In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
from bs4 import BeautifulSoup

Importing Dataframe and making it readable.

In [76]:
url = 'https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
#soup.prettify

Locating the data

In [28]:
table = soup.find('table')
data = soup.findAll('td')
rows = soup.findAll('tr')

Cleaning up the data into a table I can use

In [63]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)


canada_df = pd.DataFrame(data = data,columns = columns)
canada_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Changing Borough name to Neighbourhood where needed and dropping values where Borough is not assigned

In [71]:
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

canada_df = canada_df.drop_duplicates()
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park
...,...,...,...
254,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
261,M4Y,Downtown Toronto,Church and Wellesley
264,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
265,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [73]:
canada_df.shape

(103, 3)

In [88]:
coord_csv = pd.read_csv('http://cocl.us/Geospatial_data')
coord_csv

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [89]:
frames = [canada_df, coord_csv]
new_frame=pd.concat(frames, sort=False)
new_frame

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
2,M3A,North York,Parkwoods,NaN,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN,NaN
4,M5A,Downtown Toronto,Harbourfront,NaN,NaN,NaN
5,M6A,North York,"Lawrence Heights, Lawrence Manor",NaN,NaN,NaN
7,M7A,Downtown Toronto,Queen's Park,NaN,NaN,NaN
...,...,...,...,...,...,...
98,NaN,NaN,NaN,M9N,43.706876,-79.518188
99,NaN,NaN,NaN,M9P,43.696319,-79.532242
100,NaN,NaN,NaN,M9R,43.688905,-79.554724
101,NaN,NaN,NaN,M9V,43.739416,-79.588437
